##Topic modeling - LDA model on Corona virus text data with Apache Spark and Spark NLP

 Install Java, PySpark, and Spark NLP

In [ ]:
import os
# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version
# Install pyspark
! pip install --ignore-installed pyspark==3.5.3 #Updated
# Install Spark NLP
! pip install --ignore-installed spark-nlp==5.3.2 #Updated


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
openjdk version "1.8.0_432"
OpenJDK Runtime Environment (build 1.8.0_432-8u432-ga~us1-0ubuntu2~22.04-ga)
OpenJDK 64-Bit Server VM (build 25.432-bga, mixed mode)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 17.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=8bab2f3458171e0dc43bf1b66a70faf7a3f8bfb040b1d3592867c630a5b314bf
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 9.4 MB/s eta 

 Import the relevant package

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
spark = sparknlp.start()
! python -V
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


Python 3.10.12
Spark NLP version:  5.3.2
Apache Spark version:  3.5.3


#LDA model on Corona virus text data



To find topics in the coronavirus tweets dataset:


Also, tried
different values of k and maxIter to see which combination best suits the data.

Showed
five combinations,  their results, and explained why it’s best.

Coronavirus tweets dataset (sample)
• Dataset location (Linux filesystem): /home/data/CSC534BDA/datasets/COVID19/
i. Dataset filename: coronavirus-text-only-1000.txt
ii. Download/copy the file from cscluster
1.  use the ‘scp’ Linux command or SFTP clients like
CyberDuck or Filezilla.
iii. Upload/copy the file to your Colab so you can access the file in your Colab.
1.  use Files Explorer in Colab or Google Drive
2. For details, https://neptune.ai/blog/google-colab-dealing-with-files

• It contains 1000 tweets (text message only) with the common keyword ‘coronavirus’ in
their text messages. So ‘coronavirus’ can be considered a stopword and should be
removed.

##1. Reading file

In [ ]:
file_location= r'/content/coronavirus-text-only-1000.txt'
file_type='lineSep'
df=spark.read.option(file_type,"\n")\
.text(file_location)
df.count()


1000

Show data after reading from file and converted to data frame

In [ ]:
df.show(10)

+--------------------+
|               value|
+--------------------+
|              text\r|
|Studies look at t...|
|"RT @EricTopol: T...|
|"RT @NPR: Working...|
|"RT @Harvey_Walke...|
|RT @CNNEE: La far...|
|"RT @ReutersWorld...|
|"RT @CNN: This Il...|
|"RT @Censelio: Ar...|
|RT @jilevin: Trum...|
+--------------------+
only showing top 10 rows



##2. Pre-processing Pieline using spark-nlp

In [ ]:
document_assembler = DocumentAssembler() \
.setInputCol("value") \
.setOutputCol("document") \
.setCleanupMode("shrink")

Tokenizing data

In [ ]:
tokenizer = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token")

Normalizing data

In [ ]:
normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normalized")

Stopwords cleaning

In [ ]:
stopwords_cleaner = StopWordsCleaner()\
.setInputCols("normalized")\
.setOutputCol("cleanTokens")\
.setStopWords(["coronavirus"])\
.setCaseSensitive(False)

Stemming data

In [ ]:
stemmer = Stemmer() \
.setInputCols(["cleanTokens"]) \
.setOutputCol("stem")

Finishing the data

In [ ]:
finisher = Finisher() \
.setInputCols(["stem"]) \
.setOutputCols(["tokens"]) \
.setOutputAsArray(True) \
.setCleanAnnotations(False)

In [ ]:
nlp_pipeline = Pipeline(
stages=[document_assembler,
tokenizer,
normalizer,
stopwords_cleaner,
stemmer,
finisher])


##Train and apply ML pipeline

In [ ]:
nlp_model = nlp_pipeline.fit(df)
processed_df = nlp_model.transform(df)
tokens_df = processed_df.select('tokens').limit(10000)
tokens_df.show()

+--------------------+
|              tokens|
+--------------------+
|              [text]|
|[studi, look, at,...|
|[rt, erictopol, t...|
|[rt, npr, work, m...|
|[rt, harveywalk, ...|
|[rt, cnnee, la, f...|
|[rt, reutersworld...|
|[rt, cnn, thi, il...|
|[rt, censelio, ar...|
|[rt, jilevin, tru...|
|[rt, propublica, ...|
|[nsw, to, close, ...|
|[rt, aslavitt, tr...|
|[rt, claytravi, d...|
|[rt, jamesgunn, i...|
|[rt, natashafatah...|
|[rt, crissl, yäôa...|
|[rt, censelio, ar...|
|[rt, villarruelcl...|
|[rt, jaxalemani, ...|
+--------------------+
only showing top 20 rows



##Feature Engineering

In [ ]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="tokens", outputCol="features",
vocabSize=500, minDF=3.0)
# train the model
cv_model = cv.fit(tokens_df)
# transform the data. Output column name will be features.
vectorized_tokens = cv_model.transform(tokens_df)

## Build the LDA Model
Tried with paramters: num_topics=3, maxIterations=10

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 3
lda = LDA(k=num_topics, maxIter=10)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " +
str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -72147.6722530659
The upper bound on perplexity: 5.380942142979259


##Visualize the topics

In [ ]:
vocab = cv_model.vocabulary
topics = model.describeTopics()
topics_rdd = topics.rdd
topics_words = topics_rdd\
  .map(lambda row: row['termIndices'])\
  .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
  .collect()
for idx, topic in enumerate(topics_words):
  print("topic: {}".format(idx))
  print("*"*25)
  for word in topic:
    print(word)
print("*"*25)


topic: 0
*************************
de
rt
la
el
a
en
que
y
lo
no
topic: 1
*************************
rt
the
and
in
to
a
of
florida
their
white
topic: 2
*************************
the
rt
a
to
in
of
i
trump
and
she
*************************


##Tried with 5 more combinations in addition to the above combination as shown below:

1) number  of topics= 5, max Iterations= 10

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 5
lda = LDA(k=num_topics, maxIter=10)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " +
str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -72018.89426431859
The upper bound on perplexity: 5.37133757937937


2) number  of topics= 7, max Iterations= 20

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 7
lda= LDA(k=num_topics, maxIter=20)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " +
str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -71910.05255982341
The upper bound on perplexity: 5.363219910488023


3) number  of topics= 11, max Iterations= 23

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 11
lda= LDA(k=num_topics, maxIter=23)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " +
str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -71951.9920213258
The upper bound on perplexity: 5.3663478536191676


4) number  of topics= 15, max Iterations= 25

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 15
lda= LDA(k=num_topics, maxIter=25)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " +
str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -74662.37843135759
The upper bound on perplexity: 5.568494811407935


5) number  of topics= 18, max Iterations= 28

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 18
lda= LDA(k=num_topics, maxIter=28)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " +
str(ll))
print("The upper bound on perplexity: " + str(lp))


The lower bound on the log likelihood of the entire corpus: -77558.97745058485
The upper bound on perplexity: 5.784529941123571


num_topics =5 ,maxIter=10

gave better performance out of all the combinations tried above and resulted in higher loglikelihood and lower perplexity values of -72018.89426431859
& 5.37133757937937 respectively.